In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

from model.configs import SR
from model.utils import clean_single_wav, gen_mfcc


def load_text_metadata(texts_csv):
    df = pd.read_csv(texts_csv)
    return df["file"].tolist(), df["text"].tolist()


def normalize_fname(f_name):
    """Ensure filename has NO extension"""
    return os.path.splitext(f_name)[0]


def generate_and_cache_mfcc(wavs_dir, texts_csv, cache_dir="mfcc_cache"):

    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)

    print("Loading transcription metadata...")
    file_ids, texts = load_text_metadata(texts_csv)

    print(f"Total audio files: {len(file_ids)}")
    print("Extracting MFCC & caching...\n")

    for raw_name in tqdm(file_ids):

        # remove extension if exists
        f_name = normalize_fname(raw_name)

        output_path = os.path.join(cache_dir, f"{f_name}.npy")

        if os.path.exists(output_path):
            continue

        flac_path = os.path.join(wavs_dir, f"{f_name}.flac")

        if not os.path.exists(flac_path):
            print(f"Missing file: {flac_path}")
            continue

        try:
            wav, _ = librosa.load(flac_path, sr=SR)
        except Exception as e:
            print(f"Error loading {flac_path}: {e}")
            continue

        wav = clean_single_wav(wav)
        mfcc = gen_mfcc(wav)
        np.save(output_path, mfcc)

    print("\nMFCC feature extraction complete!")
    print(f"Saved MFCC cache to ROOT directory: {cache_dir}")


if __name__ == "__main__":
    generate_and_cache_mfcc(
        wavs_dir=r"E:\DATASET\nepali_merged",
        texts_csv=r"E:\DATASET\metadata.csv",
        cache_dir="mfcc_cache"
    )


Loading transcription metadata...
Total audio files: 2219550
Extracting MFCC & caching...



100%|██████████| 2219550/2219550 [3:04:33<00:00, 200.43it/s]      



MFCC feature extraction complete!
Saved MFCC cache to ROOT directory: mfcc_cache


In [2]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import edit_distance as ed
import matplotlib.pyplot as plt

from model.configs import SR, device_name, UNQ_CHARS, INPUT_DIM, NUM_UNQ_CHARS, MODEL_NAME
from model.utils import batchify, indices_from_texts
from model.model import get_model


# =====================================================================
#                LOAD CACHED MFCC DATA + TEXT LABELS
# =====================================================================

def load_cached_mfcc(cache_dir, texts_csv):
    df = pd.read_csv(texts_csv)

    mfcc_features = []
    texts = []

    print("Loading MFCC cache into RAM...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        f = os.path.splitext(row["file"])[0] + ".npy"
        mfcc_path = os.path.join(cache_dir, f)

        if not os.path.exists(mfcc_path):
            print("Missing MFCC:", mfcc_path)
            continue

        mfcc = np.load(mfcc_path, allow_pickle=True)
        mfcc_features.append(mfcc)
        texts.append(row["text"])

    print("Finished loading MFCC cache.")
    return mfcc_features, texts


# =====================================================================
#                           TRAINING LOOP
# =====================================================================

def train_model(
        model, optimizer,
        train_feats, train_texts,
        test_feats, test_texts,
        epochs=40, batch_size=8
    ):

    history_train_loss = []
    history_test_loss = []
    history_test_cer = []

    with tf.device(device_name):

        for e in range(epochs):
            print(f"\n========== EPOCH {e+1}/{epochs} ==========")
            start_time = time.time()

            # ---- Training ----
            train_loss = 0
            batches = 0

            for start in tqdm(range(0, len(train_feats), batch_size), desc="Training"):
                end = min(start + batch_size, len(train_feats))

                x, target, target_lengths, output_lengths =
                    batchify(train_feats[start:end], train_texts[start:end], UNQ_CHARS)

                with tf.GradientTape() as tape:
                    output = model(x, training=True)
                    loss = K.ctc_batch_cost(target, output, output_lengths, target_lengths)

                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))

                train_loss += np.mean(loss.numpy())
                batches += 1

            train_loss /= batches

            # ---- Validation ----
            test_loss = 0
            test_cer = 0
            batches = 0

            for start in tqdm(range(0, len(test_feats), batch_size), desc="Validation"):
                end = min(start + batch_size, len(test_feats))

                x, target, target_lengths, output_lengths =
                    batchify(test_feats[start:end], test_texts[start:end], UNQ_CHARS)

                output = model(x, training=False)

                # Loss
                loss = K.ctc_batch_cost(target, output, output_lengths, target_lengths)
                test_loss += np.mean(loss.numpy())

                # CER
                input_len = np.ones(output.shape[0]) * output.shape[1]
                decoded = K.ctc_decode(output, input_length=input_len,
                                       greedy=False, beam_width=100)[0][0]

                # Clean target indices
                target_clean = [t[t != 0].tolist() for t in target]

                predicted_clean = [
                    p[p > 1].numpy().tolist() for p in decoded
                ]

                batch_cer = 0
                for i in range(len(predicted_clean)):
                    sm = ed.SequenceMatcher(predicted_clean[i], target_clean[i])
                    d = sm.distance()
                    batch_cer += d / len(target_clean[i])

                test_cer += batch_cer / len(predicted_clean)
                batches += 1

            test_loss /= batches
            test_cer /= batches

            # Log
            history_train_loss.append(train_loss)
            history_test_loss.append(test_loss)
            history_test_cer.append(test_cer)

            print(f"Epoch {e+1}:  Train Loss={train_loss:.4f} | "
                  f"Val Loss={test_loss:.4f} | CER={test_cer*100:.2f}% | "
                  f"time={time.time() - start_time:.2f}s")

    return history_train_loss, history_test_loss, history_test_cer


# =====================================================================
#                     PLOT LOSS & CER CURVES
# =====================================================================

def plot_training_curves(train_loss, test_loss, test_cer):
    plt.figure(figsize=(14, 6))

    # Loss Curve
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label="Train Loss")
    plt.plot(test_loss, label="Validation Loss")
    plt.title("Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("CTC Loss")
    plt.legend()

    # CER Curve
    plt.subplot(1, 2, 2)
    plt.plot(test_cer, label="Validation CER")
    plt.title("CER Curve (%)")
    plt.xlabel("Epoch")
    plt.ylabel("CER")
    plt.legend()

    plt.tight_layout()
    plt.savefig("training_curves.png")
    plt.show()


# =====================================================================
#                           MAIN STARTS HERE
# =====================================================================

if __name__ == "__main__":

    # Load cached MFCC & texts
    feats, texts = load_cached_mfcc(
        cache_dir="mfcc_cache",
        texts_csv="E:/DATASET/metadata.csv"
    )

    # Train-test split
    train_feats, test_feats, train_texts, test_texts = train_test_split(
        feats, texts, test_size=0.2
    )

    # Model
    model = get_model(
        INPUT_DIM, NUM_UNQ_CHARS,
        num_res_blocks=5,
        num_cnn_layers=2,
        cnn_filters=50,
        cnn_kernel_size=15,
        rnn_dim=170,
        rnn_dropout=0.15,
        num_rnn_layers=2,
        num_dense_layers=1,
        dense_dim=340,
        model_name=MODEL_NAME,
        rnn_type="lstm",
        use_birnn=True
    )
    optimizer = tf.keras.optimizers.Adam()

    # Train
    tr_loss, val_loss, val_cer = train_model(
        model, optimizer,
        train_feats, train_texts,
        test_feats, test_texts,
        epochs=40,
        batch_size=8
    )

    # Plot curves
    plot_training_curves(tr_loss, val_loss, val_cer)

    # Save final model
    model.save("trained_model_from_mfcc_cache.h5")
    print("\nModel Saved Successfully!")


SyntaxError: invalid syntax (3344443682.py, line 72)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm
import edit_distance as ed
from sklearn.model_selection import train_test_split

from model.configs import SR, device_name, UNQ_CHARS, INPUT_DIM, MODEL_NAME, NUM_UNQ_CHARS
from model.utils import batchify, indices_from_texts
from model.model import get_model


class TrainingHistory:
    """Store and visualize training metrics"""
    def __init__(self):
        self.train_losses = []
        self.val_losses = []
        self.val_cers = []
        self.epochs = []
        self.train_times = []
    
    def update(self, epoch, train_loss, val_loss, val_cer, epoch_time):
        self.epochs.append(epoch)
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)
        self.val_cers.append(val_cer)
        self.train_times.append(epoch_time)
    
    def plot_curves(self, save_path="training_curves.png"):
        """Plot training and validation curves"""
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss curves
        axes[0].plot(self.epochs, self.train_losses, 'b-', label='Train Loss', linewidth=2)
        axes[0].plot(self.epochs, self.val_losses, 'r-', label='Validation Loss', linewidth=2)
        axes[0].set_xlabel('Epoch', fontsize=12)
        axes[0].set_ylabel('CTC Loss', fontsize=12)
        axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
        axes[0].legend(fontsize=10)
        axes[0].grid(True, alpha=0.3)
        
        # CER curve
        axes[1].plot(self.epochs, [cer * 100 for cer in self.val_cers], 'g-', linewidth=2)
        axes[1].set_xlabel('Epoch', fontsize=12)
        axes[1].set_ylabel('Character Error Rate (%)', fontsize=12)
        axes[1].set_title('Validation CER', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"\n📊 Training curves saved to: {save_path}")
        plt.close()
    
    def save_metrics(self, save_path="training_metrics.csv"):
        """Save metrics to CSV"""
        df = pd.DataFrame({
            'epoch': self.epochs,
            'train_loss': self.train_losses,
            'val_loss': self.val_losses,
            'val_cer': self.val_cers,
            'epoch_time_sec': self.train_times
        })
        df.to_csv(save_path, index=False)
        print(f"📝 Training metrics saved to: {save_path}")


def load_cached_mfcc(file_ids, cache_dir="mfcc_cache"):
    """Load pre-computed MFCC features from cache"""
    mfccs = []
    missing_files = []
    
    print("Loading cached MFCC features...")
    for f_name in tqdm(file_ids):
        # Remove extension if present
        f_name_clean = os.path.splitext(f_name)[0]
        cache_path = os.path.join(cache_dir, f"{f_name_clean}.npy")
        
        if not os.path.exists(cache_path):
            missing_files.append(f_name)
            continue
        
        try:
            mfcc = np.load(cache_path)
            mfccs.append(mfcc)
        except Exception as e:
            print(f"Error loading {cache_path}: {e}")
            missing_files.append(f_name)
    
    if missing_files:
        print(f"⚠️ Warning: {len(missing_files)} files missing from cache")
        print(f"First few missing: {missing_files[:5]}")
    
    print(f"✅ Loaded {len(mfccs)} MFCC features from cache\n")
    return mfccs


def compute_cer(predicted_indices, target_indices):
    """Compute Character Error Rate for a batch"""
    total_cer = 0
    valid_samples = 0
    
    for pred, truth in zip(predicted_indices, target_indices):
        if len(truth) == 0:
            continue
        
        sm = ed.SequenceMatcher(pred, truth)
        ed_dist = sm.distance()
        total_cer += ed_dist / len(truth)
        valid_samples += 1
    
    return total_cer / valid_samples if valid_samples > 0 else 0


def train_model_with_cache(model, optimizer, train_mfccs, train_texts, 
                           val_mfccs, val_texts, epochs=100, batch_size=50,
                           save_dir="model_checkpoints"):
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    history = TrainingHistory()
    best_val_cer = float('inf')
    
    with tf.device(device_name):
        
        for e in range(epochs):
            epoch_start = time.time()
            
            len_train = len(train_mfccs)
            len_val = len(val_mfccs)
            
            train_loss_total = 0
            val_loss_total = 0
            val_cer_total = 0
            
            train_batch_count = 0
            val_batch_count = 0
            
            # ============= TRAINING PHASE =============
            print(f"\n{'='*60}")
            print(f"Epoch {e+1}/{epochs} - TRAINING")
            print(f"{'='*60}")
            
            # Shuffle training data
            indices = np.random.permutation(len_train)
            train_mfccs_shuffled = [train_mfccs[i] for i in indices]
            train_texts_shuffled = [train_texts[i] for i in indices]
            
            for start in tqdm(range(0, len_train, batch_size), desc="Training"):
                end = min(start + batch_size, len_train)
                
                x, target, target_lengths, output_lengths = batchify(
                    train_mfccs_shuffled[start:end], 
                    train_texts_shuffled[start:end], 
                    UNQ_CHARS
                )
                
                with tf.GradientTape() as tape:
                    output = model(x, training=True)
                    loss = K.ctc_batch_cost(target, output, output_lengths, target_lengths)
                
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
                
                train_loss_total += np.average(loss.numpy())
                train_batch_count += 1
            
            # ============= VALIDATION PHASE =============
            print(f"\nEpoch {e+1}/{epochs} - VALIDATION")
            print(f"{'='*60}")
            
            for start in tqdm(range(0, len_val, batch_size), desc="Validation"):
                end = min(start + batch_size, len_val)
                
                x, target, target_lengths, output_lengths = batchify(
                    val_mfccs[start:end], 
                    val_texts[start:end], 
                    UNQ_CHARS
                )
                
                output = model(x, training=False)
                loss = K.ctc_batch_cost(target, output, output_lengths, target_lengths)
                
                val_loss_total += np.average(loss.numpy())
                val_batch_count += 1
                
                # Compute CER
                input_len = np.ones(output.shape[0]) * output.shape[1]
                decoded_indices = K.ctc_decode(
                    output, 
                    input_length=input_len,
                    greedy=False, 
                    beam_width=100
                )[0][0]
                
                # Process target and predicted indices
                target_indices = [sent[sent != 0].tolist() for sent in target]
                predicted_indices = [sent[sent > 1].numpy().tolist() for sent in decoded_indices]
                
                batch_cer = compute_cer(predicted_indices, target_indices)
                val_cer_total += batch_cer
            
            # ============= EPOCH SUMMARY =============
            train_loss_avg = train_loss_total / train_batch_count
            val_loss_avg = val_loss_total / val_batch_count
            val_cer_avg = val_cer_total / val_batch_count
            epoch_time = time.time() - epoch_start
            
            # Update history
            history.update(e + 1, train_loss_avg, val_loss_avg, val_cer_avg, epoch_time)
            
            print(f"\n{'='*60}")
            print(f"EPOCH {e+1} SUMMARY")
            print(f"{'='*60}")
            print(f"Train Loss:      {train_loss_avg:.4f}")
            print(f"Val Loss:        {val_loss_avg:.4f}")
            print(f"Val CER:         {val_cer_avg*100:.2f}%")
            print(f"Epoch Time:      {epoch_time:.2f} sec")
            print(f"{'='*60}\n")
            
            # Save best model
            if val_cer_avg < best_val_cer:
                best_val_cer = val_cer_avg
                best_model_path = os.path.join(save_dir, "best_model.h5")
                model.save(best_model_path)
                print(f"🎯 New best model saved! CER: {best_val_cer*100:.2f}%\n")
            
            # Save checkpoint every 5 epochs
            if (e + 1) % 5 == 0:
                checkpoint_path = os.path.join(save_dir, f"model_epoch_{e+1}.h5")
                model.save(checkpoint_path)
                print(f"💾 Checkpoint saved: {checkpoint_path}\n")
        
        # Save final model
        final_model_path = os.path.join(save_dir, "final_model.h5")
        model.save(final_model_path)
        print(f"\n✅ Final model saved: {final_model_path}")
        
        # Plot and save curves
        history.plot_curves(os.path.join(save_dir, "training_curves.png"))
        history.save_metrics(os.path.join(save_dir, "training_metrics.csv"))
        
        print(f"\n{'='*60}")
        print(f"TRAINING COMPLETE")
        print(f"{'='*60}")
        print(f"Best Validation CER: {best_val_cer*100:.2f}%")
        print(f"Total Training Time: {sum(history.train_times)/3600:.2f} hours")
        print(f"{'='*60}\n")
    
    return history


def load_data_from_cache(texts_csv, cache_dir="mfcc_cache"):
    """Load texts and corresponding cached MFCC features"""
    print("Loading metadata...")
    df = pd.read_csv(texts_csv)
    file_ids = df["file"].tolist()
    texts = df["text"].tolist()
    
    print(f"Total samples in metadata: {len(file_ids)}")
    
    # Load cached MFCCs
    mfccs = load_cached_mfcc(file_ids, cache_dir)
    
    # Ensure texts and mfccs match
    if len(mfccs) != len(texts):
        print(f"⚠️ Mismatch: {len(mfccs)} MFCCs vs {len(texts)} texts")
        # Align them (assuming order is preserved)
        min_len = min(len(mfccs), len(texts))
        mfccs = mfccs[:min_len]
        texts = texts[:min_len]
    
    print(f"✅ Final dataset size: {len(mfccs)} samples\n")
    return mfccs, texts


if __name__ == "__main__":
    
    print("\n" + "="*60)
    print("ASR MODEL TRAINING WITH CACHED MFCC FEATURES")
    print("="*60 + "\n")
    
    # ============= MODEL DEFINITION =============
    print("Defining model architecture...")
    model = get_model(
        INPUT_DIM, NUM_UNQ_CHARS, 
        num_res_blocks=5, 
        num_cnn_layers=2,
        cnn_filters=50, 
        cnn_kernel_size=15, 
        rnn_dim=170, 
        rnn_dropout=0.15, 
        num_rnn_layers=2,
        num_dense_layers=1, 
        dense_dim=340, 
        model_name=MODEL_NAME, 
        rnn_type="lstm",
        use_birnn=True
    )
    print("✅ Model defined successfully\n")
    
    # ============= OPTIMIZER =============
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    # ============= LOAD DATA =============
    mfccs, texts = load_data_from_cache(
        texts_csv=r"E:\DATASET\metadata.csv",
        cache_dir="mfcc_cache"
    )
    
    # ============= TRAIN-TEST SPLIT =============
    print("Splitting data into train and validation sets...")
    train_mfccs, val_mfccs, train_texts, val_texts = train_test_split(
        mfccs, texts, test_size=0.05, random_state=42
    )
    
    print(f"Training samples:   {len(train_mfccs)}")
    print(f"Validation samples: {len(val_mfccs)}\n")
    
    # ============= TRAIN MODEL =============
    history = train_model_with_cache(
        model=model,
        optimizer=optimizer,
        train_mfccs=train_mfccs,
        train_texts=train_texts,
        val_mfccs=val_mfccs,
        val_texts=val_texts,
        epochs=60,
        batch_size=32,
        save_dir="model_checkpoints"
    )
    
    print("\n🎉 Training pipeline completed successfully! 🎉\n")

c:\Users\AJIT\miniconda3\envs\asr_env\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.22.3)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top
c:\Users\AJIT\miniconda3\envs\asr_env\lib\site-packages\llvmlite\binding\ffi.py:175: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename



ASR MODEL TRAINING WITH CACHED MFCC FEATURES

Defining model architecture...
✅ Model defined successfully

Loading metadata...
Total samples in metadata: 2219550
Loading cached MFCC features...


  4%|▍         | 94374/2219550 [18:45<21:28:39, 27.49it/s] 